In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Url'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                title = html.xpath('//h1[@itemprop="name"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                src = html.xpath('//div[@class="productView-img-container"]/a/@href')[0].strip()

                # = = = = = = = = = = = = = = =
                
                list_dt =html.xpath('//dl[@class="productView-info"]/dt/text()')
                list_dd = html.xpath('//dl[@class="productView-info"]/dd/text()')

                # = = = = = = = = = = = = = = =
                                
                list_tr = html.xpath('//div[@id="tabDescription-content"]/table/tbody/tr')

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Title': title,
                                         'Pic': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Src': src}])

                for dt, dd in zip(list_dt, list_dd):
                    df_temp.loc[0, dt.strip()[:-1].lower()] = dd.strip()

                for tr in list_tr:
                    if tr.xpath('./td[1]/text()')[0].strip() == 'Application':
                        df_temp.loc[0, tr.xpath('./td[1]/text()')[0].strip()] = '\n'.join([vehicle.strip() for vehicle in tr.xpath('./td[2]/p/text()')])
                    else:
                        df_temp.loc[0, tr.xpath('./td[1]/text()')[0].strip()] = ';'.join([td.strip() for td in tr.xpath('./td[2]/text()')])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Url'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：355

[ok] - https://rotomaster.com/products/performance-upgrades/performance-upgrade-turbo-1998-1999-ford-7-3l-billet-wheel-w-o-pedestal-a1380153n/
[尝试次数：1] - [剩余数量：345] - [当前时间：21:35:12]

[ok] - https://rotomaster.com/turbochargers/reman-turbo-for-1980-1983-38l-buick-a8030113r/
[尝试次数：1] - [剩余数量：344] - [当前时间：21:35:12]

[ok] - https://rotomaster.com/products/performance-upgrades/performance-upgrade-turbo-1999-2003-ford-stage-2-w-66mm-88-mm-compressor-wheel-7-3l-a1380105n/
[尝试次数：1] - [剩余数量：343] - [当前时间：21:35:12]

[ok] - https://rotomaster.com/turbochargers/reman-turbo-for-1980-1981-38l-chevrolet-a8300102r/
[尝试次数：1] - [剩余数量：342] - [当前时间：21:35:13]

[ok] - https://rotomaster.com/turbochargers/reman-turbo-for-1983-1984-26-mitsubishi-m8050107r/
[尝试次数：1] - [剩余数量：341] - [当前时间：21:35:13]

[ok] - https://rotomaster.com/products/performance-upgrades/performance-upgrade-turbo-1999-2003-7-3l-w-billet-wheel-ford-a1380152n/
[尝试次数：1] - [剩余数量：340] - [当前时间：21:35:13]

[ok] - https://rotomaster.com/prod

[ok] - https://rotomaster.com/turbochargers/reman-turbo-for-2000-2002-59l-cummins-automatic-trans-dodge-h8350109r/
[尝试次数：1] - [剩余数量：288] - [当前时间：21:35:33]

[ok] - https://rotomaster.com/turbochargers/reman-turbo-for-1998-1999-73l-ford-a8380103r/
[尝试次数：1] - [剩余数量：287] - [当前时间：21:35:33]

[ok] - https://rotomaster.com/turbochargers/reman-turbo-for-1999-2001-18l-volkswagen-k8030102r/
[尝试次数：1] - [剩余数量：286] - [当前时间：21:35:33]

[ok] - https://rotomaster.com/turbochargers/reman-turbo-for-1999-2005-20l-23l-saab-m8040186r/
[尝试次数：1] - [剩余数量：285] - [当前时间：21:35:34]

[ok] - https://rotomaster.com/turbochargers/reman-turbo-for-1999-2008-87l-76l-dt530-dt466-ic-international-a8400118r/
[尝试次数：1] - [剩余数量：284] - [当前时间：21:35:34]

[ok] - https://rotomaster.com/turbochargers/reman-turbo-for-2000-2002-59l-cummins-manual-trans-dodge-h8350108r/
[尝试次数：1] - [剩余数量：283] - [当前时间：21:35:35]

[ok] - https://rotomaster.com/turbochargers/reman-turbo-for-2000-2003-30l-saab-a8150104r/
[尝试次数：1] - [剩余数量：282] - [当前时间：21:35:35]

[ok] - https://rotomaster.com/turbochargers/reman-turbo-for-2005-2007-28l-jeep-a8200104r/
[尝试次数：1] - [剩余数量：230] - [当前时间：21:35:54]

[ok] - https://rotomaster.com/turbochargers/reman-turbo-for-2007-2012-67l-dodge-ram-h8350112r/
[尝试次数：1] - [剩余数量：229] - [当前时间：21:35:54]

[ok] - https://rotomaster.com/turbochargers/reman-turbo-for-2007-2012-20l-23l-acura-m8040167r/
[尝试次数：1] - [剩余数量：228] - [当前时间：21:35:54]

[ok] - https://rotomaster.com/turbochargers/reman-turbo-for-2007-2012-30l-jeep-mercedes-benz-a8200103r/
[尝试次数：1] - [剩余数量：227] - [当前时间：21:35:54]

[ok] - https://rotomaster.com/turbochargers/reman-turbo-for-2007-2016-20l-buick-chevrolet-pontiac-k8040140r/
[尝试次数：1] - [剩余数量：226] - [当前时间：21:35:55]

[ok] - https://rotomaster.com/turbochargers/reman-turbo-for-2007-2013-30l-rear-bmw-m8030102r/
[尝试次数：1] - [剩余数量：225] - [当前时间：21:35:55]

[ok] - https://rotomaster.com/turbochargers/reman-turbo-for-2007-2016-30l-front-bmw-m8030101r/
[尝试次数：1] - [剩余数量：224] - [当前时间：21:35:56]

[ok] - https://rotomaster.com/t

[ok] - https://rotomaster.com/turbochargers/replacement-turbo-for-1977-1985-180l-tl8106-a1810115n/
[尝试次数：1] - [剩余数量：171] - [当前时间：21:36:20]

[ok] - https://rotomaster.com/turbochargers/reman-turbo-with-performance-billet-wheel-for-1999-2003-73l-ford-a8380108r/
[尝试次数：1] - [剩余数量：170] - [当前时间：21:36:21]

[ok] - https://rotomaster.com/turbochargers/replacement-turbo1975-1982-7-6l-john-deere-a1410103n/
[尝试次数：1] - [剩余数量：169] - [当前时间：21:36:21]

[ok] - https://rotomaster.com/products/turbochargers/replacement-turbo-for-1986-2003-cummins-6bt-5-9l-h1010102n/
[尝试次数：1] - [剩余数量：168] - [当前时间：21:36:21]

[ok] - https://rotomaster.com/turbochargers/replacement-turbo-06a145704s-2001-2004-volkswagen-beetle/
[尝试次数：1] - [剩余数量：167] - [当前时间：21:36:22]

[ok] - https://rotomaster.com/turbochargers/replacement-turbo1981-1984-7-6l-john-deere-a1410104n/
[尝试次数：1] - [剩余数量：166] - [当前时间：21:36:23]

[ok] - https://rotomaster.com/turbochargers/replacement-turbo1989-1991-5-9l-dodge-cummins-h1350135n/
[尝试次数：1] - [剩余数量：165] -

[ok] - https://rotomaster.com/turbochargers/replacement-turbo2000-2005-2-7l-left-audi-k1030159n/
[尝试次数：1] - [剩余数量：114] - [当前时间：21:36:41]

[ok] - https://rotomaster.com/turbochargers/replacement-turbo2000-2013-15-2l-caterpillar-c15-acert-high-pressure-a1420115n/
[尝试次数：1] - [剩余数量：113] - [当前时间：21:36:41]

[ok] - https://rotomaster.com/turbochargers/replacement-turbo2000-2002-5-9l-auto-trans-dodge-cummins-h1350103n/
[尝试次数：1] - [剩余数量：112] - [当前时间：21:36:42]

[ok] - https://rotomaster.com/turbochargers/replacement-turbo2002-2002-1-8l-audi-tt-quattro-k1040199n/
[尝试次数：1] - [剩余数量：111] - [当前时间：21:36:42]

[ok] - https://rotomaster.com/turbochargers/replacement-turbo2000-2004-2-4l-oil-feed-line-outlets-volvo-m1040123n/
[尝试次数：1] - [剩余数量：110] - [当前时间：21:36:42]

[ok] - https://rotomaster.com/turbochargers/reman-turbo-for-2012-2013-20l-bmw-m8040128r/
[尝试次数：1] - [剩余数量：109] - [当前时间：21:36:43]

[ok] - https://rotomaster.com/turbochargers/replacement-turbo2002-2008-2-5l-2-0l-saab-subaru-m1040199n/
[尝试次数：1] -

[ok] - https://rotomaster.com/turbochargers/turbocharger-audi-a3-quattro-tt-quattro-vw-beetle-cc-eos-gti-jetta-passat-tiguan-k1030171n/
[尝试次数：1] - [剩余数量：56] - [当前时间：21:37:01]

[ok] - https://rotomaster.com/turbochargers/replacement-turbo2009-2012-1-6l-mini-cooper-k1030172n/
[尝试次数：1] - [剩余数量：55] - [当前时间：21:37:02]

[ok] - https://rotomaster.com/turbochargers/replacement-turbo2009-2014-2-0l-audi-volkswagen-k1430104n/
[尝试次数：1] - [剩余数量：54] - [当前时间：21:37:02]

[ok] - https://rotomaster.com/products/turbochargers/replacement-turbo-for-2009-2018-freightliner-medium-heavy-duty-15-0l-h1450118n/
[尝试次数：1] - [剩余数量：53] - [当前时间：21:37:02]

[ok] - https://rotomaster.com/products/turbochargers/replacement-turbo-for-2010-2018-kenworth-heavy-duty-15-0l-h1450114n/
[尝试次数：1] - [剩余数量：52] - [当前时间：21:37:03]

[ok] - https://rotomaster.com/products/turbochargers/replacement-turbo-for-2008-2018-mack-medium-duty-12-8l-h1430111n/
[尝试次数：1] - [剩余数量：51] - [当前时间：21:37:03]

[ok] - https://rotomaster.com/turbochargers/turb

[ok] - https://rotomaster.com/turbochargers/replacement-turbo-for-caterpillar-t1237-cat-6n6606-a1120113n/
[尝试次数：1] - [剩余数量：0] - [当前时间：21:37:26]

[ok] - https://rotomaster.com/turbochargers/replacement-turbo-for-komatsu-excavator-ktr130-9g2-q1130102n/
[尝试次数：1] - [剩余数量：0] - [当前时间：21:37:26]

[ok] - https://rotomaster.com/turbochargers/replacement-turbo-for-man-truck-k29-2004-2006-k1290101n/
[尝试次数：1] - [剩余数量：0] - [当前时间：21:37:27]

[ok] - https://rotomaster.com/turbochargers/replacement-turbo-for-komatsu-ktr130-11f-d150a-eg300-q1130101n/
[尝试次数：1] - [剩余数量：0] - [当前时间：21:37:29]

[ok] - https://rotomaster.com/pedestals/turbo-mount-for-1998-1999-ford-7-3l-a1382207n/
[尝试次数：1] - [剩余数量：0] - [当前时间：21:37:30]

[ok] - https://rotomaster.com/turbochargers/turbocharger-cadillac-ats-ct6-cts-chevy-camaro-malibu-equinox-gmc-terrain-m1040106n/
[尝试次数：2] - [剩余数量：0] - [当前时间：21:38:24]

[ok] - https://rotomaster.com/turbochargers/replacement-turbo2013-2021-3-5l-left-ford-lincoln-ecoboost-k1030132n/
[尝试次数：2] - [剩余数